In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import os
# os.chdir('/content/drive/MyDrive/DAIV_2024_Fall/PromptMRG')
# %pwd

'/content/drive/MyDrive/DAIV_2024_Fall/PromptMRG'

In [11]:
#!git clone https://github.com/Hyunnji-Lee/PromptMRG.git

# medpix_data json파일 만들기

In [1]:
import json

Descriptions_file_path = 'Descriptions.json'
with open(Descriptions_file_path, 'r', encoding='utf-8') as f:
    descriptions_data = json.load(f)

Case_file_path = 'Case_topic.json'
with open(Case_file_path, 'r', encoding='utf-8') as f:
    case_data = json.load(f)

In [2]:
# 흉부 데이터 거르기 (334개)
filtered_descriptions = [data for data in descriptions_data if data['Location Category']=='Thorax']

### 질병 labels 관련

In [4]:
filtered_cases = []
id = [description['U_id'] for description in filtered_descriptions]

for case in case_data:
  if case['U_id'] in id:
      filtered_cases.append(case)

In [6]:
print(len(filtered_cases))

132


In [7]:
ex_description = filtered_cases[0]
ex_findings = [ex_description["Case"]["Findings"]]
print(ex_findings)

['•  PA chest radiograph demonstrates left lung volume loss, silhouetting of the left cardiac border, and Luftsichel sign.\n\n•  Lateral chest radiograph shows anterior displacement of the major fissure and elevation of the left mainstem bronchus.\n\n•  CT of the chest reveals an obstructing mass and resultant LUL collapse.']


In [13]:
from modules.chexbert import CheXbert
import torch

checkpoint = r'C:\Users\user1\Desktop\DAIV_2024_Fall\PromptMRG\checkpoints\stanford\chexbert\chexbert.pth'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
chexbert = CheXbert(checkpoint_path=checkpoint, device=device)
output = chexbert(ex_findings)
print(output)

tensor([[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])


#### case 내 질병 확인용

In [49]:
from collections import Counter

title = [case['Topic']['Title'] for case in filtered_cases]
Counter(title)

Counter({'Lung, lobar collapse (left upper lobe)': 1,
         'Adenocarcinoma of the Lung': 1,
         'Interstital Lung Disease': 1,
         'Pericardial cyst': 1,
         'Idiopathic Pulmonary Hemosiderosis': 1,
         'Pulmonary Embolism': 3,
         'thoracic aortic dissection': 1,
         'Bronchial Carcinoid': 2,
         "Takayasu's Arteritis": 1,
         'Pulmonary Hamartoma': 2,
         'Lung Metastasis': 1,
         'Sarcoid': 1,
         'Pyopneumothorax': 1,
         'Metastatic Adenocarcinoma of the Lung': 1,
         'Swyer-James Syndrome': 1,
         'Reactivation TB': 1,
         'Bronchiectasis - Bronchial Artery Embolization.': 1,
         'Pancoast tumor': 1,
         'Ascending thoracic aortic aneurysm': 1,
         'Pulmonary Embolus': 1,
         'Left aortic arch with aberrant right subclavian artery': 3,
         'Coarctation of the Aorta': 2,
         'Benign Metastasizing Leiomyoma': 1,
         'Cryptococcal Pneumonia in an Immunocompetent Host': 1

In [51]:
count = list(Counter(title))
len(count)

121

### clip indices 관련

In [4]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl (273 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Cloning https://github.com/openai/CLIP.git to c:\users\user1\appdata\local\temp\pip-req-build-3h88mg76
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.20.1-cp312-cp312-win_amd64.whl.metadata (6.2 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Usi

  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\user1\AppData\Local\Temp\pip-req-build-3h88mg76'


In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

tensor([[ 0.0547, -0.0061,  0.0495,  ..., -0.6638, -0.1281, -0.4950],
        [ 0.1447,  0.0225, -0.2909,  ..., -0.4472, -0.3420,  0.1798],
        [ 0.1981, -0.2040, -0.1533,  ..., -0.4514, -0.5664,  0.0596]])


In [ ]:
print(text_features.shape)

torch.Size([3, 512])


In [ ]:
import clip
import torch
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)
text_features = text_embedding

image_path = '/content/drive/MyDrive/DAIV_2024_Fall/PromptMRG/data/medpix/images/MPX1007_synpic46719.png'
image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(21)

print(indices)

tensor([ 30446,  30447, 195450, 195449,  15025,  15026,  37231,  37232, 262662,
        262663, 125199, 125200,  10208,  10207,  10206, 195458, 140842, 140841,
         78025,  78024,  78022])


### json 파일 만들기

In [3]:
import json
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [4]:
# clip_text_feature 가져오기
clip_text_feature_path = '../mimic_cxr/clip_text_features.json'
with open(clip_text_feature_path, 'r', encoding='utf-8') as f:
    clip_text_feature = json.load(f)
text_features = torch.tensor(clip_text_feature)

In [5]:
import sys
import os

sys.path.append(r'C:\Users\user1\Desktop\DAIV_2024_Fall\PromptMRG')

from modules.chexbert import CheXbert
import torch

checkpoint = r'C:\Users\user1\Desktop\DAIV_2024_Fall\PromptMRG\checkpoints\stanford\chexbert\chexbert.pth'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
chexbert = CheXbert(checkpoint_path=checkpoint, device=device)

C:\Users\user1\Desktop\DAIV_2024_Fall\PromptMRG\modules\chexbert.py:51: SyntaxWarning: invalid escape sequence '\s'
  reports[i] = reports[i].replace("\s+", " ")
C:\Users\user1\Desktop\DAIV_2024_Fall\PromptMRG\modules\chexbert.py:52: SyntaxWarning: invalid escape sequence '\s'
  reports[i] = reports[i].replace("\s+(?=[\.,])", "")
c:\Users\user1\Desktop\DAIV_2024_Fall\PromptMRG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\user1\Desktop\DAIV_2024_Fall\PromptMRG\modules\chexbert.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-

In [87]:
import re

all_data = []

for data in filtered_descriptions:
    each_data = {}
    
    # id & image_id
    id = data['U_id']
    image_id = data['image']

    each_data['id'] = id
    
    # type
    each_data['type'] = data['Type']
    
    # image_path
    each_data['image_path'] = []
    image_path = data['image'] + '.png'
    each_data['image_path'].append(image_path)

    # report
    findings = [case["Case"]["Findings"] for case in case_data if case["U_id"] == id]
    report = findings[0]
    
    # 불필요한 문자 제거
    report = report.replace("\n", " ")
    report = report.replace("\t", " ")
    report = report.replace('\u201c', '').replace('\u201d', '').replace('\u2022', '')
    report = re.sub(r'\s+', ' ', report)
        
    each_data['report'] = report

    # clip_indices
    image_path = './images/' + image_id + '.png'

    # image encoding
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)

    # Pick the top 21 most similar labels for the image
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity[0].topk(21)
    
    each_data['clip_indices'] = indices.tolist()
    
    # split
    each_data['split'] = 'test'

    # labels
    labels = chexbert(findings).tolist()
    each_data['labels'] = labels[0]
    
    all_data.append(each_data)

In [88]:
print(len(all_data))

334


In [89]:
print(all_data[0])

{'id': 'MPX1024', 'type': 'CT', 'image_path': ['MPX1024_synpic40275.png'], 'report': ' PA chest radiograph demonstrates left lung volume loss, silhouetting of the left cardiac border, and Luftsichel sign. Lateral chest radiograph shows anterior displacement of the major fissure and elevation of the left mainstem bronchus. CT of the chest reveals an obstructing mass and resultant LUL collapse.', 'clip_indices': [15408, 15409, 67504, 67505, 67503, 3807, 3808, 11418, 11419, 10370, 10371, 134561, 151593, 268516, 268517, 26647, 26646, 260156, 171033, 171032, 26339], 'split': 'test', 'labels': [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]}


In [90]:
file_data = all_data[:3]
print(json.dumps(file_data, ensure_ascii=False, indent='\t'))

[
	{
		"id": "MPX1024",
		"type": "CT",
		"image_path": [
			"MPX1024_synpic40275.png"
		],
		"report": " PA chest radiograph demonstrates left lung volume loss, silhouetting of the left cardiac border, and Luftsichel sign. Lateral chest radiograph shows anterior displacement of the major fissure and elevation of the left mainstem bronchus. CT of the chest reveals an obstructing mass and resultant LUL collapse.",
		"clip_indices": [
			15408,
			15409,
			67504,
			67505,
			67503,
			3807,
			3808,
			11418,
			11419,
			10370,
			10371,
			134561,
			151593,
			268516,
			268517,
			26647,
			26646,
			260156,
			171033,
			171032,
			26339
		],
		"split": "test",
		"labels": [
			0,
			0,
			0,
			1,
			0,
			0,
			0,
			1,
			0,
			0,
			0,
			0,
			0,
			0
		]
	},
	{
		"id": "MPX1024",
		"type": "CT",
		"image_path": [
			"MPX1024_synpic40277.png"
		],
		"report": " PA chest radiograph demonstrates left lung volume loss, silhouetting of the left cardiac border, and Luftsichel sign

In [94]:
with open('medpix_annotation_promptmrg.json', 'w', encoding='utf-8') as make_file:
    json.dump(all_data, make_file, ensure_ascii=False, indent='\t')